In [1]:
#!pip install selenium
#!pip install selenium-wire
#!pip install beautifulsoup4
#!pip install lxml

In [1]:
#scraper class to initialize standard features

#constants to load extensions and define the path for screenshots

EXTENSION_PATH_NT = r'G:\rat\rat\templates\I-don-t-care-about-cookies.crx'
EXTENSION_PATH_LINUX = '//I-don-t-care-about-cookies.crx'

SCREENSHOT_PATH_NT = r'G:\rat\rat\templates\tmp\\'
SCREENSHOT_PATH_LINUX = "//tmp//"

import json
import base64
from bs4 import BeautifulSoup

from urllib import request
from urllib.parse import urlsplit
from urllib.parse import urlparse
import urllib.parse
import socket

import os
import inspect

import uuid #used to generate random file names

import time

class Scraping:

    def __init__(self):
        self = self

    def __del__(self):
        print('Helper object destroyed')

    def encode_code(self, code):
        code = code.encode('utf-8','ignore')
        code = base64.b64encode(code)
        return code

    def decode_code(self, value):

        try:
            code_decoded = base64.b64decode(value)
            code_decoded = BeautifulSoup(code_decoded, "html.parser")
            code_decoded = str(code_decoded)
        except Exception as e:
            print(str(e))
            code_decoded = "decoding error"
        return code_decoded



    def decode_picture(self, value):
        picture = value.tobytes()
        picture = picture.decode('ascii')
        return picture

    def get_result_meta(self, url):
        meta = {}
        ip = "-1"
        main = url
        #parse url to get hostname and socket
        try:
            parsed_uri = urlparse(url)
            hostname = '{uri.netloc}'.format(uri=parsed_uri)
            ip = socket.gethostbyname(hostname)
        except Exception as e:
            print(str(e))
            ip = "-1"

        try:
            main = '{0.scheme}://{0.netloc}/'.format(urlsplit(url))
        except Exception as e:
            print(str(e))
            main = url

        #write to meta dictionary
        meta = {"ip":ip, "main":main}

        return meta

    def get_chrome_extension(self):
        #change extension path to load the I-don't care about cookies file
        
        currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
        parentdir = os.path.dirname(currentdir)
        
        #check for the used operating system to load the correct folder structure
        if os.name == "nt":
            extension_path = EXTENSION_PATH_NT
        else:
            extension_path = EXTENSION_PATH_LINUX
        return extension_path


    def take_screenshot(self, driver):

        #function to encode file content to base64
        def encode_file_base64(self, file):
            f = open(file, 'rb')
            code = f.read()
            code = base64.b64encode(code)
            f.close()
            return code

        current_path = os.path.abspath(os.getcwd())

        #iniatilize constant variables

        #iniatilize the directories for the extension and for the folder for temporary downlods of files
        
        #check for the used operating system to load the correct folder structure
        if os.name == "nt":
            screenshot_folder = SCREENSHOT_PATH_NT


        else:
            screenshot_folder = SCREENSHOT_PATH_LINUX

        screenshot_file = screenshot_folder+str(uuid.uuid1())+".png"
              
        time.sleep(2)

        driver.maximize_window() #maximize browser window for screenshot
        
        try:
            driver.execute_script("window.scrollTo(0,1)")
        except Exception as e:
            print(str(e))
            pass

        #try to get the whole browser window
        try: 
            required_width = driver.execute_script('return document.body.parentNode.scrollWidth')
            required_height = driver.execute_script('return document.body.parentNode.scrollHeight')
                      
            scroll = "window.scrollTo(0,{})".format(required_height)
            
            driver.execute_script(scroll)

            required_height+= 50
            
            driver.execute_script("window.scrollTo(0,1)")
            
            driver.set_window_size(required_width, required_height)
            
            driver.maximize_window()
            
            driver.save_screenshot(screenshot_file) #take screenshot
            
        except Exception as e:
            print(str(e)) ##next try based on the document.body

            try: 
                body_screenshot = driver.find_element(By.TAG_NAME, "body")
                body_screenshot.screenshot(screenshot_file)
            except Exception as e:
                print(str(e)) #if all fails take screenshot of the browser view
                driver.save_screenshot(screenshot_file) #take screenshot

        #open screenshot and save as base64
        screenshot = encode_file_base64(self, screenshot_file)

        #os.remove(screenshot_file) #remove the comment if you don't want to store the screenshots

        return screenshot #return base64 code of image

    #function to read out redirected urls

    def get_real_url(url, driver):
                
        try:
            driver.get(url)
            time.sleep(4)
            current_url = driver.current_url #read real url (redirected url)
            driver.quit()
            return current_url
        except Exception as e:
            print(str(e))
            pass
        


In [3]:
"""
This template describes the steps required to add a custom scraper for the RAT software. First of all, it is assumed that these are search services that provide search forms. However, it is also possible to add other search systems. The procedure would have to be adapted accordingly. Selenium is used as the basis for scraping.

A scraper generally consists of the following functions:
- run(query, limit, scraping): Main function for all scrapers with the following parameters: query = search query, limit = maximum number of results to be retrieved, scraping = scraping object with functions for scraping the search engines
- get_search_results(driver, page): Sub-function for retrieving the search results with the following parameters:
driver = Selenium driver; web browser for scraping; page = SERP page
- check_captcha(driver): Helper function to check whether there is a block on search services. The function is called to indicate that scraping has been cancelled. If the check is false, scraping is continued.

All standard variables and functions for scraping a search engine are described below. However, it is also possible to change everything here according to the search engine to be scraped.

Link to the documentation on finding elements on a webpage using Selenium: https://selenium-python.readthedocs.io/locating-elements.html

"""

#required libs for web scraping

#import external libraries
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.common.exceptions import TimeoutException #used to interrupt loding of websites and needed as workaround to download files with selenium
from selenium.webdriver.common.action_chains import ActionChains #used to simulate pressing of a key

import uuid #used to generate random file names

import time #used to do timeout breaks

import os #used for file management

#base64 encoding to convert the code codes of webpages
import base64

#BeautifulSoup is necessary to beautify the code coded after it has been decoded (especially useful to prevent character errors)
from bs4 import BeautifulSoup
from lxml import html

import random

import re

#main function to run a scraper

def run(query, limit, scraping):
    try:
        #Definition of args for scraping the search engine
        search_url = "https://www.google.de" #URL of search engine, e. g. www.google.de
        search_box = "q" #Class name of search box; input field for searches
        captcha = "g-recaptcha" #Source code hint for CAPTCHA; some search engines use CAPTCHAS to block too many automatic requests
        next_page = "//a[@aria-label='{}']" #CSS to find click on next SERP; for search engines that use a navigation on SERPS to continue browsing more search results
        results_number = 0 #initialize results_number; normally starts at 0
        page = 1 #initialize SERP page; normally starts at 1
        search_results = [] #initialize search_results list

        #Definition of custom functions
        
        def search_pagination(source):
            soup = BeautifulSoup(source, features="lxml")
            if soup.find("span", class_=["SJajHc NVbCr"]):
                return True
            else:
                return False
            
        #Function to scrape search results
        def get_search_results(driver, page):

            get_search_results = []

            source = driver.page_source

            serp_code = scraping.encode_code(source)

            serp_bin = scraping.take_screenshot(driver)

            soup = BeautifulSoup(source, features="lxml")
            
            #addtional steps to extract undesired elements from the Search Engine Result Page (SERP)

            for s in soup.find_all("div", class_="d4rhi"):
                s.extract()

            for s in soup.find_all("div", class_="Wt5Tfe"):
                s.extract()

            for s in soup.find_all("div", class_="UDZeY fAgajc OTFaAf"):
                s.extract()
                
            #find the list with the search results by extracting the div container

            for result in soup.find_all("div", class_=["tF2Cxc"]):
                url_list = []
                search_result = []
                result_title = ""
                result_description = ""
                result_url = ""
                
                #find result title of a search result by header class
                try:
                    for title in result.find("h3", class_=["LC20lb MBeuO DKV0Md"]):
                        result_title+=title.text.strip()
                except:
                    result_title = "N/A"
                    
                
                #find description of a search result by div container
                try:                  
                    for description in result.find("div", class_=re.compile("VwiC3b", re.I)):
                       
                        result_description+=description.text.strip()
                except:
                    result_description = "N/A"
                    
                #find url of a search result by href
                try:
                    for url in result.find_all("a"):
                        url = url.attrs['href']
                        url_list.append(url)
                        result_url = url_list[0]
                except:
                    result_url = "N/A"
                
                #add search results to list
                get_search_results.append([result_title, result_description, result_url, serp_code, serp_bin, page])

            return get_search_results

        #Function to check if search engine shows CAPTCHA code
        def check_captcha(driver):
            source = driver.page_source
            if captcha in source:
                return True
            else:
                return False

        chrome_extension = scraping.get_chrome_extension() #Get Path for I don't care about cookies extension

        #initialize Selenium
        options = Options()
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument("--start-maximized")
        options.add_argument('--headless=new')
        options.add_argument("--lang=de")
        options.add_experimental_option("detach", True)
        options.add_extension(chrome_extension)
        driver = webdriver.Chrome(options=options)
        driver.set_page_load_timeout(20)
        driver.implicitly_wait(30)
        driver.get(search_url)
        driver.maximize_window()
        random_sleep = random.randint(2, 5)
        time.sleep(random_sleep)

    
        #Start scraping if no CAPTCHA
        if not check_captcha(driver):

            search = driver.find_element(By.NAME, search_box)
            search.send_keys(query)
            search.send_keys(Keys.RETURN)

            random_sleep = random.randint(2, 5)
            time.sleep(random_sleep)

            search_results = get_search_results(driver, page)

            results_number = len(search_results)

            continue_scraping = True

            """
            Custom block to click on the next SERP page:
            
            Google offers two different ways to get more results on a SERP.  Either by clicking on a button for "more results" or by the classic scrolling on the SERP to subsequent pages. This customized block first checks which type of pagination is offered. 
            Subsequent search results are then scraped as long as the total number of search results is less than the defined limit.
            """
            
            check_pagination = search_pagination(source = driver.page_source)

            if check_pagination: 
                #Click on next SERP pages as long the total number of results is lower the results limit
                while (results_number < limit) and continue_scraping:
                    if not check_captcha(driver):
                        random_sleep = random.randint(2, 5)
                        time.sleep(random_sleep)
                        page+=1
                        page_label = "Page "+str(page)
                        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                        try:
                            next = driver.find_element(By.XPATH, next_page.format(page_label))
                            next.click()
                            search_results+= get_search_results(driver, page)
                            results_number = len(search_results)
                        except:
                            continue_scraping = False
                    else:
                        continue_scraping = False
                        search_results = -1

                driver.quit()

                return search_results

            else:
                SCROLL_PAUSE_TIME = 1

                while (results_number < limit) and continue_scraping:

                    if not check_captcha(driver):
                        try:
                            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                            time.sleep(SCROLL_PAUSE_TIME)
                            driver.execute_script("return document.body.scrollHeight") + 400
                            page+=1
                            search_results+= get_search_results(driver, page)
                            results_number = len(search_results)
                        except:
                            continue_scraping = False
                    else:
                        continue_scraping = False
                        search_results = -1

                
                driver.quit()    
                return search_results


        else:
            search_results = -1
            driver.quit()

    except Exception as e:
        print(str(e))
        try:
            driver.quit()
        except:
            pass
        search_results = -1
        return search_results


In [4]:
#function to test the scraper. it shows all scraped details or an error message, if it fails

def test_scraper(query, limit, scraper):
    search_results = run(query, limit, scraper)

    i = 0
    if search_results != -1:
        for sr in search_results:
            i+=1
            print(i)
            print(sr[0])
            print(sr[1])
            print(sr[2])
    else:
        print("Scraping failed")

In [5]:
#initialise the scraper: Change the parameters for testing your scraper
scraper = Scraping() #initialize the scraping object

query = "politik" #search query
limit = 10 #max_number of results (the scraper normally adds some more pages since not all search engines deliver a certain number of search results on every SERP)



In [6]:
test_scraper(query, limit, scraper)

1
Politik
Politik bezeichnet die Strukturen (Polity), Prozesse (Politics) und Inhalte (Policy) zur Regelung der Angelegenheiten eines Gemeinwesens – etwa eines ...
https://de.wikipedia.org/wiki/Politik
2
Politik - Aktuelle Nachrichten & News
vor 1 Stunde —Nachrichten aus dem In- & Ausland: Politik-News, Reportagen, Analysen, Hintergrundberichte und Interviews aus Deutschland, der Europäischen Union und der ...
https://www.welt.de/politik/
3
Aktuelle Nachrichten aus der Politik auf einen Blick
vor 14 Stunden —Politik – Nachrichten, Kommentare und Videos zur Politik in Deutschland und der Welt. Die wichtigsten Informationen zu Wahlen und dem Bundestag.
https://www.zdf.de/nachrichten/politik
4
Politik - DER SPIEGEL
vor 14 Stunden —Deutschlands führende Nachrichtenseite. Alles Wichtige aus Politik, Wirtschaft, Sport, Kultur, Wissenschaft, Technik und mehr.
https://www.spiegel.de/politik/
5
Politik | bpb.de
Politik. Allg.: P. bezeichnet jegliche Art der Einflussnahme und Gestaltung sowie di